In [2]:
# first, test generative model on one dimensional graphs
import sys
sys.path.append('./HCM') # outdated due to abstraction learning implementation
import pandas as pd
import seaborn as sns
import numpy as np
from Learning import *
from CG1 import *

In [24]:
import pandas as pd
df = pd.read_csv('/Users/swu/Desktop/research/motif_learning/data/simonsays_ed/data.csv')

dfm = {}  # model dataframe
dfm['blockcollect'] = []
dfm['ID'] = []
dfm['condition'] = []
dfm['correctcollect'] = []
dfm['p'] = []
dfm['trialcollect'] = []
dfm['recall_likelihood'] = []


column_names = ['ID', 'condition', 'recallcollect', 'instructioncollect','trialcollect', 'correctcollect']
dfs = pd.DataFrame(columns=column_names)  # Note that there are now row data inserted.


seql = 12
len_train = 40
len_test = 8
def convert_sequence(seq):
    seq = list(seq)
    x = seq[0]
    proj_seq = [] # pause
    for item in seq:
        if item == x:
            proj_seq.append(1)
        else:
            proj_seq.append(2)
    return proj_seq


def convert_sequence_backward_to_key(seq, inst_seq):
    X = ins_seq.tolist()[0]# the first occuring item 
    Y = list(set(ins_seq.tolist()).difference(X))[0]
    recall_seq = []
    for i in range(0, 12):
        if int(seq[i,0,0]) == 1:
            recall_seq.append(X)
        else:
            recall_seq.append(Y)
    return recall_seq

def calculate_prob(chunk_record, cg):
    p = 1
    for key in list(chunk_record.keys()):# key is the encoding time
        p = p*cg.chunks[chunk_record[key][0][0]].count/np.sum([item.count for item in cg.chunks])
    return p

for sub in np.unique(list(df['ID'])):
    # initialize chunking part with specified parameters
    cg = CG1(DT=0.1, theta=0.996)
    for trial in range(1, len_train + 3*len_test+ 1):
        ins_seq = df[(df['ID'] == sub)].iloc[(trial-1)*seql:trial*seql, :][
            'instructioncollect']
        subj_recall = df[(df['ID'] == sub)].iloc[(trial-1)*seql:trial*seql, :]['recallcollect']
        condition = list(df[(df['ID'] == sub)].iloc[(trial-1)*seql:trial*seql, :][
            'condition'])[0]
        block = list(df[(df['ID'] == sub)].iloc[(trial-1)*seql:trial*seql, :][
            'blockcollect'])[0]

        proj_seq = convert_sequence(ins_seq)

        if condition == 'm1':
            proj_seq = [1,2,2,2, 2,2,1,1, 1,1,2,1]
        elif condition == 'm2':
            proj_seq = [1,1,1,2, 2,1,1,2, 2,2,2,1]
        else:
            proj_seq = convert_sequence(ins_seq)

        proj_seq = np.array(proj_seq).reshape([-1, 1, 1])
    
        cg, chunkrecord = hcm_learning(proj_seq, cg)  # with the rational chunk models, rational_chunk_all_info(seq, cg)
        recalled_seq,ps = recall(cg, firstitem=proj_seq[0,0,0])
        keypress_recalled_seq = convert_sequence_backward_to_key(recalled_seq, ins_seq)

        p_seq = np.prod(ps)# evaluate the probability of a sequence

        proj_recalled_seq = convert_sequence(list(subj_recall))
        proj_recalled_seq = np.array(proj_recalled_seq).reshape([-1, 1, 1])
        cg, chunkrecord = hcm_learning(proj_recalled_seq, cg, learn = False, chunk = False)  # with the rational chunk models, rational_chunk_all_info(seq, cg)
        recall_likelihood = calculate_prob(chunkrecord, cg)# evaluate the probability of a sequence
        
        
        dfm['blockcollect'].append(block)
        dfm['ID'].append(sub)
        dfm['condition'].append(condition)
        dfm['correctcollect'].append(acc_eval1d(recalled_seq, proj_seq))
        dfm['p'].append(p_seq)
        dfm['trialcollect'].append(trial)
        dfm['recall_likelihood'].append(recall_likelihood)
        
        ins_list = ins_seq.tolist()
        print(keypress_recalled_seq, ins_list)
        for i in range(0, 12):
            if keypress_recalled_seq[i] == ins_list[i]:
                correctcollect = 1
            else: correctcollect = 0
            dfs = dfs.append({'ID': sub,
                            'condition': condition,
                            'recallcollect': keypress_recalled_seq[i],
                            'trialcollect': trial,
                            'instructioncollect': ins_list[i],
                            'correctcollect': correctcollect}, ignore_index=True)
        
        

dfm = pd.DataFrame.from_dict(dfm)
csv_save_directory = '/Users/swu/Desktop/research/motif_learning/data/simonsays_ed/simulation_data_model_transition_recall.csv'
dfm.to_csv(csv_save_directory, index=False, header=True)



dfs_csv_save_directory = '/Users/swu/Desktop/research/motif_learning/data/simonsays_ed/simulation_data_model_transition_recall_individualkey.csv'
dfs.to_csv(dfs_csv_save_directory, index=False, header=True)



accuracy is:  0.4166666666666667
['D', 'D', 'D', 'D', 'D', 'D', 'K', 'D', 'D', 'D', 'D', 'D'] ['D', 'D', 'D', 'K', 'K', 'D', 'D', 'K', 'K', 'K', 'K', 'D']
accuracy is:  0.5
['S', 'S', 'K', 'K', 'K', 'K', 'S', 'K', 'S', 'S', 'S', 'K'] ['S', 'S', 'S', 'K', 'K', 'S', 'S', 'K', 'K', 'K', 'K', 'S']
accuracy is:  0.8333333333333334
['D', 'D', 'D', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'D'] ['D', 'D', 'D', 'K', 'K', 'D', 'D', 'K', 'K', 'K', 'K', 'D']
accuracy is:  0.5833333333333334
['K', 'K', 'S', 'S', 'S', 'S', 'S', 'S', 'K', 'S', 'K', 'K'] ['K', 'K', 'K', 'S', 'S', 'K', 'K', 'S', 'S', 'S', 'S', 'K']
accuracy is:  0.8333333333333334
['S', 'S', 'S', 'J', 'J', 'S', 'S', 'S', 'S', 'J', 'J', 'S'] ['S', 'S', 'S', 'J', 'J', 'S', 'S', 'J', 'J', 'J', 'J', 'S']
accuracy is:  0.5
['K', 'S', 'S', 'S', 'S', 'K', 'K', 'K', 'S', 'K', 'K', 'S'] ['K', 'K', 'K', 'S', 'S', 'K', 'K', 'S', 'S', 'S', 'S', 'K']
accuracy is:  0.6666666666666666
['K', 'D', 'K', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D'] ['K', 

accuracy is:  0.4166666666666667
['L', 'F', 'F', 'F', 'F', 'F', 'L', 'L', 'L', 'L', 'F', 'F'] ['L', 'F', 'L', 'L', 'F', 'L', 'F', 'F', 'L', 'F', 'L', 'F']
accuracy is:  0.5
['F', 'F', 'L', 'L', 'F', 'F', 'L', 'L', 'L', 'L', 'F', 'F'] ['F', 'F', 'L', 'F', 'L', 'F', 'L', 'L', 'F', 'F', 'L', 'L']
accuracy is:  0.3333333333333333
['L', 'L', 'F', 'F', 'L', 'L', 'F', 'F', 'F', 'F', 'L', 'L'] ['L', 'F', 'L', 'F', 'F', 'F', 'L', 'L', 'L', 'F', 'F', 'L']
accuracy is:  0.25
['F', 'F', 'L', 'L', 'F', 'L', 'F', 'F', 'F', 'L', 'L', 'F'] ['F', 'L', 'F', 'F', 'L', 'F', 'L', 'F', 'L', 'F', 'L', 'L']
accuracy is:  0.8333333333333334
['L', 'L', 'L', 'F', 'F', 'L', 'L', 'F', 'F', 'F', 'F', 'L'] ['L', 'L', 'L', 'F', 'F', 'L', 'L', 'F', 'F', 'L', 'F', 'F']
accuracy is:  0.4166666666666667
['L', 'F', 'F', 'F', 'F', 'F', 'L', 'L', 'L', 'L', 'F', 'F'] ['L', 'L', 'F', 'L', 'F', 'L', 'F', 'L', 'F', 'F', 'L', 'F']
accuracy is:  0.5833333333333334
['F', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L'] ['F',

accuracy is:  0.5833333333333334
['F', 'F', 'F', 'J', 'F', 'J', 'J', 'F', 'J', 'J', 'F', 'F'] ['F', 'F', 'F', 'J', 'J', 'F', 'F', 'J', 'J', 'J', 'J', 'F']
accuracy is:  0.5833333333333334
['J', 'J', 'J', 'F', 'F', 'F', 'F', 'F', 'J', 'J', 'J', 'J'] ['J', 'J', 'J', 'F', 'F', 'J', 'J', 'F', 'F', 'F', 'F', 'J']
accuracy is:  0.6666666666666666
['F', 'J', 'F', 'F', 'J', 'F', 'J', 'J', 'J', 'F', 'J', 'F'] ['F', 'F', 'F', 'J', 'J', 'F', 'F', 'J', 'J', 'J', 'J', 'F']
accuracy is:  0.5
['J', 'F', 'J', 'F', 'F', 'J', 'F', 'F', 'J', 'J', 'J', 'F'] ['J', 'J', 'J', 'F', 'F', 'J', 'J', 'F', 'F', 'F', 'F', 'J']
accuracy is:  0.9166666666666666
['F', 'F', 'F', 'J', 'J', 'F', 'F', 'J', 'J', 'J', 'F', 'F'] ['F', 'F', 'F', 'J', 'J', 'F', 'F', 'J', 'J', 'J', 'J', 'F']
accuracy is:  0.75
['J', 'J', 'J', 'F', 'J', 'J', 'F', 'J', 'F', 'F', 'F', 'J'] ['J', 'J', 'J', 'F', 'F', 'J', 'J', 'F', 'F', 'F', 'F', 'J']
accuracy is:  0.5
['F', 'J', 'J', 'J', 'J', 'J', 'F', 'F', 'F', 'F', 'J', 'F'] ['F', 'F', 'F', 'J',

accuracy is:  1.0
['K', 'K', 'K', 'S', 'S', 'K', 'K', 'S', 'S', 'S', 'S', 'K'] ['K', 'K', 'K', 'S', 'S', 'K', 'K', 'S', 'S', 'S', 'S', 'K']
accuracy is:  0.6666666666666666
['S', 'S', 'K', 'K', 'S', 'S', 'K', 'K', 'K', 'K', 'S', 'S'] ['S', 'S', 'S', 'K', 'K', 'S', 'S', 'K', 'K', 'K', 'K', 'S']
accuracy is:  0.5833333333333334
['K', 'K', 'S', 'K', 'K', 'K', 'K', 'K', 'S', 'S', 'K', 'K'] ['K', 'K', 'K', 'S', 'S', 'K', 'K', 'S', 'S', 'S', 'S', 'K']
accuracy is:  0.75
['S', 'S', 'S', 'K', 'S', 'S', 'S', 'S', 'S', 'K', 'K', 'S'] ['S', 'S', 'S', 'K', 'K', 'S', 'S', 'K', 'K', 'K', 'K', 'S']
accuracy is:  0.5833333333333334
['K', 'K', 'K', 'K', 'K', 'S', 'S', 'K', 'K', 'K', 'K', 'S'] ['K', 'S', 'S', 'K', 'K', 'S', 'K', 'S', 'K', 'K', 'S', 'S']
accuracy is:  0.5
['K', 'K', 'K', 'S', 'S', 'K', 'K', 'K', 'K', 'S', 'S', 'K'] ['K', 'K', 'K', 'S', 'K', 'S', 'S', 'S', 'S', 'S', 'K', 'K']
accuracy is:  0.5
['S', 'S', 'K', 'K', 'S', 'S', 'K', 'K', 'K', 'K', 'S', 'S'] ['S', 'S', 'K', 'K', 'K', 'S', 'S',

accuracy is:  0.5833333333333334
['D', 'D', 'J', 'J', 'J', 'D', 'D', 'J', 'J', 'D', 'J', 'J'] ['D', 'D', 'J', 'D', 'J', 'D', 'J', 'D', 'D', 'J', 'J', 'J']
accuracy is:  0.3333333333333333
['D', 'F', 'D', 'D', 'F', 'D', 'F', 'D', 'F', 'F', 'F', 'D'] ['D', 'D', 'F', 'D', 'D', 'F', 'D', 'F', 'D', 'F', 'F', 'F']
accuracy is:  0.5
['S', 'S', 'J', 'S', 'S', 'J', 'S', 'J', 'S', 'S', 'J', 'S'] ['S', 'J', 'S', 'J', 'S', 'J', 'J', 'S', 'S', 'J', 'J', 'S']
accuracy is:  0.6666666666666666
['J', 'J', 'J', 'J', 'D', 'D', 'J', 'J', 'J', 'D', 'D', 'J'] ['J', 'J', 'J', 'D', 'D', 'J', 'J', 'J', 'D', 'D', 'D', 'D']
accuracy is:  0.5
['S', 'S', 'F', 'S', 'F', 'F', 'S', 'F', 'S', 'S', 'F', 'F'] ['S', 'F', 'S', 'S', 'F', 'S', 'S', 'F', 'F', 'F', 'S', 'F']
accuracy is:  0.6666666666666666
['L', 'L', 'K', 'L', 'L', 'K', 'L', 'K', 'L', 'K', 'K', 'K'] ['L', 'L', 'K', 'K', 'L', 'K', 'K', 'K', 'L', 'K', 'L', 'L']
accuracy is:  0.4166666666666667
['L', 'L', 'L', 'L', 'K', 'K', 'L', 'K', 'L', 'K', 'L', 'K'] ['L', 

...

accuracy is:  0.9166666666666666
['K', 'K', 'K', 'D', 'D', 'K', 'K', 'D', 'D', 'D', 'K', 'K'] ['K', 'K', 'K', 'D', 'D', 'K', 'K', 'D', 'D', 'D', 'D', 'K']
accuracy is:  1.0
['L', 'L', 'L', 'K', 'K', 'L', 'L', 'K', 'K', 'K', 'K', 'L'] ['L', 'L', 'L', 'K', 'K', 'L', 'L', 'K', 'K', 'K', 'K', 'L']
accuracy is:  1.0
['J', 'J', 'J', 'L', 'L', 'J', 'J', 'L', 'L', 'L', 'L', 'J'] ['J', 'J', 'J', 'L', 'L', 'J', 'J', 'L', 'L', 'L', 'L', 'J']
accuracy is:  1.0
['J', 'J', 'J', 'L', 'L', 'J', 'J', 'L', 'L', 'L', 'L', 'J'] ['J', 'J', 'J', 'L', 'L', 'J', 'J', 'L', 'L', 'L', 'L', 'J']
accuracy is:  0.6666666666666666
['D', 'D', 'K', 'K', 'D', 'D', 'D', 'K', 'K', 'D', 'D', 'D'] ['D', 'D', 'D', 'K', 'K', 'D', 'D', 'K', 'K', 'K', 'K', 'D']
accuracy is:  0.5
['L', 'L', 'L', 'L', 'L', 'L', 'K', 'K', 'L', 'L', 'K', 'K'] ['L', 'L', 'L', 'K', 'K', 'L', 'L', 'K', 'K', 'K', 'K', 'L']
accuracy is:  0.9166666666666666
['D', 'D', 'D', 'K', 'K', 'D', 'D', 'K', 'K', 'K', 'K', 'K'] ['D', 'D', 'D', 'K', 'K', 'D', 'D', 

accuracy is:  0.4166666666666667
['D', 'D', 'D', 'D', 'J', 'J', 'J', 'J', 'J', 'D', 'D', 'D'] ['D', 'J', 'J', 'J', 'J', 'J', 'D', 'D', 'D', 'D', 'J', 'D']
accuracy is:  0.5833333333333334
['J', 'J', 'J', 'D', 'D', 'D', 'D', 'D', 'J', 'J', 'J', 'J'] ['J', 'D', 'D', 'D', 'D', 'D', 'J', 'J', 'J', 'J', 'D', 'J']
accuracy is:  1.0
['K', 'D', 'D', 'D', 'D', 'D', 'K', 'K', 'K', 'K', 'D', 'K'] ['K', 'D', 'D', 'D', 'D', 'D', 'K', 'K', 'K', 'K', 'D', 'K']
accuracy is:  1.0
['S', 'D', 'D', 'D', 'D', 'D', 'S', 'S', 'S', 'S', 'D', 'S'] ['S', 'D', 'D', 'D', 'D', 'D', 'S', 'S', 'S', 'S', 'D', 'S']
accuracy is:  1.0
['S', 'D', 'D', 'D', 'D', 'D', 'S', 'S', 'S', 'S', 'D', 'S'] ['S', 'D', 'D', 'D', 'D', 'D', 'S', 'S', 'S', 'S', 'D', 'S']
accuracy is:  1.0
['K', 'S', 'S', 'S', 'S', 'S', 'K', 'K', 'K', 'K', 'S', 'K'] ['K', 'S', 'S', 'S', 'S', 'S', 'K', 'K', 'K', 'K', 'S', 'K']
accuracy is:  1.0
['S', 'D', 'D', 'D', 'D', 'D', 'S', 'S', 'S', 'S', 'D', 'S'] ['S', 'D', 'D', 'D', 'D', 'D', 'S', 'S', 'S', 'S', 

accuracy is:  0.5
['J', 'J', 'J', 'S', 'S', 'J', 'J', 'S', 'J', 'J', 'J', 'S'] ['J', 'S', 'S', 'S', 'J', 'J', 'S', 'J', 'J', 'J', 'S', 'S']
accuracy is:  0.5833333333333334
['K', 'K', 'K', 'S', 'S', 'K', 'K', 'S', 'S', 'S', 'K', 'K'] ['K', 'S', 'K', 'S', 'K', 'S', 'K', 'K', 'S', 'S', 'S', 'K']
accuracy is:  0.5833333333333334
['L', 'J', 'J', 'J', 'L', 'L', 'J', 'J', 'L', 'L', 'J', 'J'] ['L', 'J', 'J', 'L', 'L', 'J', 'L', 'L', 'L', 'J', 'J', 'J']
accuracy is:  0.5833333333333334
['L', 'L', 'L', 'J', 'J', 'J', 'L', 'J', 'L', 'J', 'J', 'J'] ['L', 'J', 'J', 'L', 'J', 'J', 'L', 'L', 'L', 'J', 'L', 'J']
accuracy is:  0.5
['J', 'J', 'J', 'J', 'J', 'J', 'K', 'K', 'K', 'J', 'J', 'K'] ['J', 'J', 'J', 'K', 'K', 'K', 'J', 'K', 'K', 'J', 'K', 'J']
accuracy is:  0.6666666666666666
['J', 'J', 'S', 'S', 'J', 'J', 'S', 'J', 'S', 'S', 'J', 'S'] ['J', 'J', 'S', 'S', 'J', 'J', 'S', 'S', 'S', 'J', 'S', 'J']
accuracy is:  0.6666666666666666
['L', 'L', 'L', 'S', 'S', 'S', 'L', 'S', 'S', 'S', 'L', 'L'] ['L', 

accuracy is:  0.9166666666666666
['L', 'J', 'J', 'J', 'J', 'J', 'L', 'L', 'L', 'L', 'J', 'J'] ['L', 'J', 'J', 'J', 'J', 'J', 'L', 'L', 'L', 'L', 'J', 'L']
accuracy is:  0.5
['J', 'J', 'L', 'J', 'J', 'J', 'J', 'J', 'L', 'J', 'L', 'L'] ['J', 'L', 'L', 'L', 'L', 'L', 'J', 'J', 'J', 'J', 'L', 'J']
accuracy is:  0.3333333333333333
['L', 'L', 'L', 'L', 'J', 'L', 'J', 'J', 'J', 'L', 'J', 'J'] ['L', 'J', 'J', 'J', 'J', 'J', 'L', 'L', 'L', 'L', 'J', 'L']
accuracy is:  0.5833333333333334
['D', 'D', 'L', 'D', 'L', 'L', 'L', 'L', 'D', 'D', 'D', 'D'] ['D', 'L', 'L', 'L', 'L', 'L', 'D', 'D', 'D', 'D', 'L', 'D']
accuracy is:  0.6666666666666666
['F', 'J', 'J', 'J', 'J', 'J', 'F', 'J', 'J', 'J', 'J', 'J'] ['F', 'J', 'J', 'J', 'J', 'J', 'F', 'F', 'F', 'F', 'J', 'F']
accuracy is:  0.6666666666666666
['D', 'D', 'L', 'D', 'L', 'L', 'L', 'D', 'L', 'D', 'L', 'D'] ['D', 'L', 'L', 'L', 'L', 'L', 'D', 'D', 'D', 'D', 'L', 'D']
accuracy is:  0.5
['J', 'J', 'J', 'J', 'D', 'J', 'J', 'D', 'J', 'D', 'D', 'J'] ['J', 

accuracy is:  0.5
['S', 'S', 'S', 'K', 'K', 'S', 'S', 'K', 'K', 'K', 'K', 'S'] ['S', 'K', 'K', 'S', 'K', 'S', 'K', 'K', 'S', 'S', 'K', 'S']
accuracy is:  0.5833333333333334
['K', 'S', 'K', 'K', 'S', 'S', 'S', 'K', 'K', 'S', 'S', 'S'] ['K', 'K', 'S', 'K', 'S', 'S', 'K', 'K', 'S', 'S', 'K', 'S']
accuracy is:  0.75
['K', 'S', 'S', 'K', 'K', 'S', 'S', 'S', 'K', 'K', 'S', 'S'] ['K', 'K', 'K', 'K', 'S', 'S', 'S', 'S', 'K', 'K', 'S', 'S']
accuracy is:  0.5833333333333334
['S', 'S', 'S', 'S', 'S', 'K', 'K', 'K', 'S', 'S', 'K', 'K'] ['S', 'S', 'S', 'K', 'K', 'S', 'S', 'K', 'S', 'K', 'K', 'K']


In [5]:
recalled_seq

array([[[1.]],

       [[2.]],

       [[2.]],

       [[2.]],

       [[2.]],

       [[2.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[2.]],

       [[2.]]])

In [11]:
X = ins_seq.tolist()[0]
Y = list(set(ins_seq.tolist()).difference(X))[0]

In [12]:
print(X,Y)

S K


In [10]:
ins_seq

92916    S
92917    S
92918    S
92919    K
92920    K
92921    S
92922    S
92923    K
92924    S
92925    K
92926    K
92927    K
Name: instructioncollect, dtype: object